In [1]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML(""))
import scienceplots
plt.style.use(['science', 'no-latex'])
import folium
from folium import plugins
from folium.plugins import HeatMap

In [7]:
# import
df = pd.read_csv("la_incl_coordinates.csv", parse_dates=["start_time","end_time"],low_memory=False)

In [10]:
df = df.drop('Unnamed: 0.1', axis=1)
df = df.drop('Unnamed: 0', axis=1)
df.head(3)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,date,month,weekday,day,hour,route,start_coords,end_coords
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway,7.0,2019-01-01,1,1,1,0,2nd & Hill to 7th & Broadway,"(34.05287, -118.24749)",NaN
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,6.0,2019-01-01,1,1,1,0,2nd & Hill to 7th & Broadway,"(34.05287, -118.24749)",NaN
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,2019-01-01,1,1,1,0,Main & 1st to Broadway & 9th,"(34.05194, -118.24353)","(34.04211, -118.25619)"


In [11]:
# group stations into three areas
def fun_startarea(start_station_name):
    if 'Lankershim & Califa' in start_station_name:
        return 'North Hollywood'
    elif 'Downtown Santa Monica E Line Station' in start_station_name:
        return 'Beach'
    else:
        return 'City Center'
    
df['Start Area'] = df.apply(lambda x: fun_startarea(x['start_station_name']),axis=1)

df.head(3)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,date,month,weekday,day,hour,route,start_coords,end_coords,Start Area
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway,7.0,2019-01-01,1,1,1,0,2nd & Hill to 7th & Broadway,"(34.05287, -118.24749)",NaN,City Center
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,6.0,2019-01-01,1,1,1,0,2nd & Hill to 7th & Broadway,"(34.05287, -118.24749)",NaN,City Center
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,2019-01-01,1,1,1,0,Main & 1st to Broadway & 9th,"(34.05194, -118.24353)","(34.04211, -118.25619)",City Center


In [14]:
# group stations into three areas
def fun_endarea(end_station_name):
    if 'Lankershim & Califa' or 'Chandler & Laurel Canyon' or 'Chandler & Colfax' or 'North Hollywood Station' or 'Chandler & Cahuenga' or 'Magnolia & Cartwright' or 'Magnolia & Clybourn' or 'Lankershim & Hesby' or 'Lankershim & Vineland' or 'Riverside & Cahuenga' or 'Mariota & Riverside' in end_station_name:
        return 'North Hollywood'
    elif 'Downtown Santa Monica E Line Station' in end_station_name:
        return 'Beach'
    else:
        return 'City Center'
    
df['End Area'] = df.apply(lambda x: fun_endarea(x['end_station_name']),axis=1)

df.head(3)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,date,month,weekday,day,hour,route,start_coords,end_coords,Start Area,End Area
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway,7.0,2019-01-01,1,1,1,0,2nd & Hill to 7th & Broadway,"(34.05287, -118.24749)",NaN,City Center,North Hollywood
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,6.0,2019-01-01,1,1,1,0,2nd & Hill to 7th & Broadway,"(34.05287, -118.24749)",NaN,City Center,North Hollywood
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,2019-01-01,1,1,1,0,Main & 1st to Broadway & 9th,"(34.05194, -118.24353)","(34.04211, -118.25619)",City Center,North Hollywood
